I included a few cells showing the output from using python profiling on my fit_gains function, which is included as well. If you are interested in learning more about this function, please refer to the Functions.py file which contains the full code. You can also visit CorrCal_notebook_perturbed_array.ipynb which executes this function and shows the results.

In [15]:
def fit_gains(m, runs, src_array, visibilities):
    '''
    Computes the recovered gains for both amplitude and phase. The inputs include the source
    vector, S. The two options for the vector depend on whether or not source positions are
    known.
    The vector, R, constructed from the visibility covariance, and the visibilities. The two
    options for the vector depend on whether or not you are using the redundant case.
    The visibilities input here are noiseless. Noise is input into the visibilities in this
    function, and visibilities are then separated into real and imaginary components. I either
    use the sum of unresolved and bright sources, or just the unresolved sources.

    Parameters
    ----------
    runs : int
        Number of noise realisations you want to do (500 runs is sufficient)
    src_array : ndarray
        The source vector, S (2*Nsources, 2*Nbls)
    vecs : ndarray
        The covariance vector, R (2*Nvec, 2*Nbls)
    visibilities : ndarray
        Noiseless visibilities (Nbls,)

    Returns
    -------
    recovered gains : ndarray
        Recovered gains separated in real (amplitude) and imaginary (phase) components (2*Ndish)
    '''

    t = m.telescope
    Nfeeds,_= t.feedpositions.shape
    Ndish = np.int(Nfeeds/2)
    Nbls = np.int(Ndish*(Ndish-1)/2)

    bl_arr_dish_indices, sum_counts, bl_counts = Bls_counts(m)
    lims = np.append(0, sum_counts) #these are the edges of the redundant blocks

    ant1 = bl_arr_dish_indices[:,0].astype(int)
    ant2 = bl_arr_dish_indices[:,1].astype(int)

    gain = np.ones(Ndish)
    sim_gains = np.ones(2*Ndish)
    sim_gains[::2] = gain.real
    sim_gains[1::2] = gain.imag

    S_max =  5 * 12 * 1.e-6 * np.sqrt(1024./Ndish)
    sigma = S_max*0.01 # we include a small amount of per visibility noise
    diag = sigma**2*np.ones(2*Nbls)
    random_gain = np.random.normal(0,1.e-3, 2*Ndish) #initial guess input into corrcal
    gvec = sim_gains + random_gain

    get_chisq = corrcal.get_chisq
    get_gradient = corrcal.get_gradient
    mat = corrcal.Sparse2Level(diag,vecs,src_array,2*lims)

    rec_gains = np.zeros((runs,Ndish*2))
    for ind_run in range(runs):
        print (ind_run)
        Noise_array = np.random.normal(0, sigma, 2*Nbls)
        data = np.zeros(2*visibilities.size)
        data[0::2] = visibilities.real + Noise_array[::2]
        data[1::2] = visibilities.imag + Noise_array[1::2]
        fac=1.;
        normfac=1.
        results = fmin_cg(get_chisq, gvec*fac, get_gradient,(data,mat,ant1,ant2,fac,normfac))
        fit_gains_run = results/fac
        rec_gains[ind_run,:] = fit_gains_run
        print (fit_gains_run[::2])
    return rec_gains

In [19]:
import cProfile
cProfile.run('fit_gains(m, runs=1, src_array = src_total, visibilities=Vis_total)', 'stats')

0
         Current function value: 124656569656.804977
         Iterations: 5
         Function evaluations: 93
         Gradient evaluations: 81
[1.00012925 0.99973659 0.99998151 0.99964584 0.99953996 1.00074728
 0.99927694 0.99942568 1.00066944 0.99994539 1.00129493 1.00025684
 0.99938363 1.00124587 0.99975859 1.00037691 0.99952509 1.00127568
 1.00078384 1.00034719 0.99980786 1.00006831 1.00049979 1.00063606
 1.00011115 0.99934774 1.00016453 0.99958667 0.9996931  1.00078677
 1.00014117 0.99946632 0.99892552 0.99913449 0.99983668 0.99844027
 1.00067755 0.9991614  1.0000803  0.99954854 0.99980713 0.99893143
 0.99961464 0.99918413 1.00000978 0.99904419 1.00003219 1.00025093
 0.9999414  1.00097566 0.99923702 1.0005037  1.00067581 0.99948055
 1.00027877 0.99953937 0.99768835 1.00149792 1.00068131 1.0010978
 1.00023179 0.99969347 0.99950699 0.99924325]


In [20]:
import pstats
from pstats import SortKey
p = pstats.Stats('stats')
p.strip_dirs().sort_stats(-1).print_stats()

Thu Aug 13 13:32:15 2020    stats

         2539634 function calls (2539570 primitive calls) in 22.518 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   22.518   22.518 <ipython-input-15-61c2d9300576>:1(fit_gains)
        1    0.000    0.000   22.518   22.518 <string>:1(<module>)
      112    0.015    0.000    0.024    0.000 LogCal_code.py:17(index_find)
        1    0.200    0.200    0.240    0.240 LogCal_code.py:26(Bls_counts)
   170238    0.366    0.000    0.366    0.000 __init__.py:489(cast)
      174    0.001    0.000    0.003    0.000 _dtype.py:319(_name_get)
   170238    0.049    0.000    0.049    0.000 _internal.py:257(__init__)
   170238    0.986    0.000    0.986    0.000 _internal.py:260(__array_interface__)
   170238    0.508    0.000    3.395    0.000 _internal.py:272(_get_void_ptr)
   170238    0.232    0.000    3.626    0.000 _internal.py:289(__init__)
   170238    0.040    0.000  

In [24]:
p.sort_stats(SortKey.TIME).print_stats(50)

Thu Aug 13 13:32:15 2020    stats

         2539634 function calls (2539570 primitive calls) in 22.518 seconds

   Ordered by: internal time
   List reduced from 203 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      174    9.271    0.053   19.295    0.111 corrcal.py:125(inv)
      641    4.170    0.007    4.171    0.007 {built-in method numpy.dot}
      255    1.026    0.004    1.087    0.004 corrcal.py:79(__mul__)
   170238    0.986    0.000    0.986    0.000 _internal.py:260(__array_interface__)
   179209    0.842    0.000    1.828    0.000 {built-in method numpy.array}
     1217    0.826    0.001    0.828    0.001 {method 'astype' of 'numpy.ndarray' objects}
     1374    0.810    0.001    0.812    0.001 {method 'copy' of 'numpy.ndarray' objects}
      174    0.695    0.004    0.740    0.004 corrcal.py:209(apply_gains_to_mat)
   109446    0.581    0.000    0.806    0.000 defmatrix.py:191(__getitem__)
   170238    0.508    0.0